# Alpha Factor Evaluation

### Loading libraries

In [3]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd
import pandas_datareader.data as web

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Model Interpreter
import shap

# Warnings
import warnings

# Path, OS & Time
import os, sys
from time import time
from pathlib import Path

# StatsModel
import statsmodels.api as sm

# Scikit-Learn
from sklearn.preprocessing import scale
from sklearn.feature_selection import mutual_info_regression

# SciPy
from scipy.stats import spearmanr

# Gradient Booster
import lightgbm as lgb

In [4]:
from utils import MultipleTimeSeriesCV

In [7]:
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [9]:
idx= pd.IndexSlice

sns.set_style('whitegrid')

warnings.filterwarnings('ignore')

In [11]:
results_path = Path('results')

if not results_path.exists():
    results_path.mkdir()

### Loading Data

In [14]:
# factors = (pd.concat([pd.read_hdf('data.h5', 'factors/common'),
#                       pd.read_hdf('data.h5', 'factors/formulaic')
#                       .rename(columns=lambda x: f'alpha_{int(x):03}')],
#                      axis=1)
#            .dropna(axis=1, thresh=100000)
#            .sort_index())

In [16]:
# factors.info()

In [20]:
# fwd_returns = factors.filter(like='fwd').columns

# features = factors.columns.difference(fwd_returns).tolist()

# alphas = pd.Index([f for f in features if f.startswith('alpha')])

In [22]:
# features

In [24]:
# len(alphas)

### Factor Correlation

#### 'Classic' Factors

In [27]:
# corr_common = factors.drop(fwd_returns.union(alphas), axis=1).corr(method='spearman')

In [29]:
# corr_common.to_hdf('data.h5', 'correlation/common')

In [31]:
# mask = np.triu(np.ones_like(corr_common, dtype=np.bool))

# fig, ax = plt.subplots(figsize=(22, 18))

# cmap = sns.diverging_palette(10, 220, as_cmap=True)

# sns.heatmap(corr_common, mask=mask, cmap=cmap, center=0,
#             square=True, linewidths=.5, cbar_kws={"shrink": .5})
# fig.tight_layout()
# fig.savefig(results_path / 'factor_corr_common', dpi=300);
# plt.show()

In [33]:
# g = sns.clustermap(corr_common, cmap=cmap, figsize=(15, 15))

# g.savefig(results_path / 'factor_corr_common_cluster', dpi=300);

In [35]:
# corr_ = corr_common.stack().reset_index()

# corr_.columns = ['x1', 'x2', 'rho']
# corr_ = corr_[corr_.x1!=corr_.x2].drop_duplicates('rho')

In [37]:
# corr_.nlargest(5, columns='rho').append(corr_.nsmallest(5, columns='rho'))

#### Formulaic Alphas

In [41]:
# %%time
# corr_formula = factors[alphas].sort_index().corr(method='spearman').dropna(how='all', axis=1)
# corr_formula.to_hdf('data.h5', 'correlation/formula')

In [43]:
# corr_formula = corr_formula.dropna(how='all').dropna(how='all', axis=1)

In [45]:
# mask = np.triu(np.ones_like(corr_formula, dtype=np.bool))
# fig, ax = plt.subplots(figsize=(22, 18))
# cmap = sns.diverging_palette(10, 220, as_cmap=True)

# sns.heatmap(corr_formula, mask=mask, cmap=cmap, center=0,
#             square=True, linewidths=.5, cbar_kws={"shrink": .5})
# fig.tight_layout()
# fig.savefig(results_path / 'factor_correlation_formula', dpi=300);

In [47]:
# g = sns.clustermap(corr_formula.replace((np.inf, -np.inf), np.nan), cmap=cmap, figsize=(15, 15))
# g.savefig(results_path / 'factor_correlation_formula_cluster', dpi=300);

In [49]:
# corr_formula_ = corr_formula.stack().reset_index()
# corr_formula_.columns = ['x1', 'x2', 'rho']
# corr_formula_ = corr_formula_[corr_formula_.x1!=corr_formula_.x2].drop_duplicates('rho')

In [51]:
# corr_formula_.nlargest(5, columns='rho').append(corr_formula_.nsmallest(5, columns='rho'))

### All Factors